# Odporúčanie

In [13]:
%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# dependencies
import pandas as pd
import wandb
from dotenv import load_dotenv

from _helpers import constants
from _helpers.verify_submission.verify_subm import main as verify_subm
from _helpers.score_submission.score_subm import main as score_subm
from models.model_random import ModelRandom

In [15]:
# Load env variables from .env file
load_dotenv()

False

Nainicializujeme Weights and Biases (logovací nástroj pre strojové učenie)

In [16]:
wandb.login()

True

In [17]:
df_train = pd.read_csv(constants.TRAIN)

Spustíme trénovanie

In [18]:
models = {
    'random': ModelRandom(),
}

In [19]:
run = 1
notes = ''
params = {
    'model': 'random',
    'seed': 12345
}

wandb_run = wandb.init(entity='mcfreddie777', project="dp-recsys", name=f'model_{params["model"]}_run_{run}', notes=notes)
wandb_run.config.update(params)

model = models[params['model']]
model.update(params)
model.fit(df_train)

Predikujeme odporúčania

In [20]:
df_test = pd.read_csv(constants.TEST)
df_recommendations = model.predict(df_test)

Verifikujeme predikcie

In [21]:
verify_subm(df_subm=df_recommendations,df_test=df_test)

Checking for required columns in the submission file...
> check passed
Checking for duplicate sessions in the submission file...
> check passed
Checking that all the required sessions are present in submission...
> check passed
All checks passed


In [22]:
df_recommendations.to_csv(constants.OUTPUT_DIR / f'submission_popular_{params["model"]}_{run}.csv', index=False)

Vypočítame si MRR na dátach

In [23]:
df_gt = pd.read_csv(constants.GROUND_TRUTH)
mrr,map3 = score_subm(df_subm=df_recommendations,df_gt=df_gt)
wandb_run.log({"mrr":mrr, "map3":map3})

Mean reciprocal rank:      0.1788
Mean average precision @3: 0.052


In [24]:
wandb_run.finish()


map3,▁
mrr,▁
map3,0.052
mrr,0.1788
